## Imports and data preparation


In [2]:
import matplotlib.pyplot as plt
import math
import numpy as np
import skimage.io as io
from skimage.util import img_as_float, img_as_uint
from skimage.transform import rotate
from skimage.transform import EuclideanTransform
from skimage.transform import SimilarityTransform
from skimage.transform import warp
from skimage.transform import swirl
from skimage.transform import matrix_transform

## Geometric transformations on images

- Rotation with and without anchor point
- Euclidean image transformation (Rotate and traslate)
- Similarly image transformation (Rotate, traslate and scale)
- Swirl image transformation (make a twist in the image)


In [1]:
def show_comparison(original, transformed, transformed_name):
    fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(8, 4), sharex=True, sharey=True)
    ax1.imshow(original)
    ax1.set_title("Original")
    ax1.axis("off")
    ax2.imshow(transformed)
    ax2.set_title(transformed_name)
    ax2.axis("off")
    io.show()

### Image rotation


In [ ]:
# Image rotation NO anchor point

rotation_angle = 10
rotated_img = rotate(im_org, rotation_angle)
show_comparison(im_org, rotated_img, "Rotated image")

In [ ]:
# Image rotation WITH anchor point

rot_center = [0, 0]
rotated_img = rotate(im_org, rotation_angle, center=rot_center)
show_comparison(im_org, rotated_img, "Rotated image")

In [ ]:
# Image rotation wiht mode
# mode="reflect" -> riflette la parte più vicina come sfondo
# mode="constant", cval=1  -> mette un colore a piacere (1=bianco)
# mode="wrap"  -> mette la parte opposta dell'immagine

rotated_img = rotate(im_org, rotation_angle, mode="reflect")
show_comparison(im_org, rotated_img, "Rotated image")

In [ ]:
# resize="True" -> lascia invaraite le dimensioni della foto, quindi aumenta il quadrato che la circonda

rotated_img = rotate(im_org, rotation_angle, resize=True)
show_comparison(im_org, rotated_img, "Rotated image")

### Euclidean image transformation


In [ ]:
# angle in radians - counter clockwise
rotation_angle = 10.0 * math.pi / 180.0
trans = [10, 20]
# EuclideanTransform returns a matrix with parameters for euclidean transformation. ROTATION AND TRASLATION
tform = EuclideanTransform(rotation=rotation_angle, translation=trans)
print(tform.params)

# deformation of the image, warp applies the tform (linear in this case) to the image
transformed_img = warp(im_org, tform)
show_comparison(im_org, transformed_img, "Rigid transformation image")

### Similarity transform of image


In [ ]:
rotation_angle = 15.0 * math.pi / 180.0
trans = [40, 30]
scale = 0.6
# Same as the Euclidean but with ROTATION, TRASLATION AND SCALE
tform = SimilarityTransform(scale=scale, rotation=rotation_angle, translation=trans)

recovered_img = warp(transformed_img, tform)
show_comparison(im_org, recovered_img, "Similarity transformation")

### The swirl image transformation


In [ ]:
str = 10
rad = 300
c = [500, 400]
# Swirl make a spiral in the immage with these parameters
swirl_img = swirl(im_org, strength=str, radius=rad, center=c)
show_comparison(im_org, swirl_img, "Swirl transformation")

## Landmark based registration

%matplotlib widget makes you visualize the plot with pixel values

- Compare images one over the other
- Create landmarks for images
- Calculate errors between 2 images' landmarks
- Estimate best matrix with **Least Squarred Error** method


In [ ]:
# Visualize image sovrapposed
blend = 0.5 * img_as_float(src_img) + 0.5 * img_as_float(dst_img)
io.imshow(blend)
io.show()

In [ ]:
# Create a landmark array to visualize in the image
# *********src  IS THE MOVING IMAGE, dst IS THE FIXED IMAGE***************
src = np.array([[588, 274], [328, 179], [134, 398], [260, 525], [613, 448]])

plt.imshow(src_img)
plt.plot(src[:, 0], src[:, 1], ".r", markersize=12)
plt.show()

In [ ]:
# Plot a line that connects all the landmarks for each image
%matplotlib inline 

dst = np.array([[621, 293], [382, 166], [198, 266], [270, 440], [600, 450]])

fig, ax = plt.subplots()
io.imshow(blend)
ax.plot(src[:, 0], src[:, 1], '-r', markersize=12, label="Source")
ax.plot(dst[:, 0], dst[:, 1], '-g', markersize=12, label="Destination")
ax.invert_yaxis()
ax.legend()
ax.set_title("Landmarks before alignment")
plt.show()

In [ ]:
# Difference error between image's landmarks

e_x = src[:, 0] - dst[:, 0]
error_x = np.dot(e_x, e_x)
e_y = src[:, 1] - dst[:, 1]
error_y = np.dot(e_y, e_y)
f = error_x + error_y
print(f"Landmark alignment error F (sum of squared differences): {f}")

In [ ]:
# Estimate uses the "Total least squared error" method to calculate the "best" matrix to use for the transformation

tform = EuclideanTransform()
tform.estimate(src, dst)
src_transform = matrix_transform(src, tform.params)

fig, ax = plt.subplots()
io.imshow(dst_img)
ax.plot(
    src_transform[:, 0],
    src_transform[:, 1],
    "-r",
    markersize=12,
    label="Source transform",
)
ax.plot(dst[:, 0], dst[:, 1], "-g", markersize=12, label="Destination")
ax.invert_yaxis()
ax.legend()
ax.set_title("Landmarks after alignment")
plt.show()

warped = warp(src_img, tform.inverse)
show_comparison(src_img, warped, 'Landmark based transformation')
